In [1]:
# Modified from source at https://github.com/tensorpack/tensorpack/blob/master/examples/Saliency/saliency-maps.py
# Modified to rely on inception_v1 rather than resnet_v1
# Used under license

import numpy as np
import sys
from contextlib import contextmanager
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.nets import inception

import tensorpack as tp
import tensorpack.utils.viz as viz

import PIL
import tempfile
from urllib.request import urlretrieve
import tarfile
import os

IMAGE_SIZE = 224

In [2]:
@contextmanager
def guided_relu():
    """
    Returns:
        A context where the gradient of :meth:`tf.nn.relu` is replaced by
        guided back-propagation, as described in the paper:
        `Striving for Simplicity: The All Convolutional Net
        <https://arxiv.org/abs/1412.6806>`_
    """
    from tensorflow.python.ops import gen_nn_ops   # noqa

    @tf.RegisterGradient("GuidedReLU")
    def GuidedReluGrad(op, grad):
        return tf.where(0. < grad,
                        gen_nn_ops.relu_grad(grad, op.outputs[0]),
                        tf.zeros(grad.get_shape()))

    g = tf.get_default_graph()
    with g.gradient_override_map({'Relu': 'GuidedReLU'}):
        yield


def saliency_map(output, input, name="saliency_map"):
    """
    Produce a saliency map as described in the paper:
    `Deep Inside Convolutional Networks: Visualising Image Classification Models and Saliency Maps
    <https://arxiv.org/abs/1312.6034>`_.
    The saliency map is the gradient of the max element in output w.r.t input.

    Returns:
        tf.Tensor: the saliency map. Has the same shape as input.
    """
    max_outp = tf.reduce_max(output, 1)
    saliency_op = tf.gradients(max_outp, input)[:][0]
    return tf.identity(saliency_op, name=name)


class Model(tp.ModelDescBase):
    def inputs(self):
        # should effectively be same as image = tf.Variable(tf.zeros((224, 224, 3)))
        return [tf.placeholder(tf.float32, (IMAGE_SIZE, IMAGE_SIZE, 3), 'image')]

    def build_graph(self, orig_image):
        mean = tf.get_variable('inception_v1/mean_rgb', shape=[3])
        with guided_relu():
            with slim.arg_scope(inception.inception_v1_arg_scope(weight_decay=0.0)):
                image = tf.expand_dims(orig_image - mean, 0)
                logits, _ = inception.inception_v1(
                    image, 1001, is_training=False, reuse=False)
                logits = logits[:,1:] # ignore background class
            saliency_map(logits, orig_image, name="saliency")
            
model_path = "/home/w266ajh/Documents/inception_v1.ckpt"
predictor = tp.OfflinePredictor(tp.PredictConfig(
        model=Model(),
        session_init=tp.get_model_loader(model_path),
        input_names=['image'],
        output_names=['saliency']))

def run(model_path, image_path):
    fname = "normal_saliencies/" + image_path.split('/')[-1]
    
    im = PIL.Image.open(image_path)

    # resnet expect RGB inputs of 224x224x3
#     im = cv2.resize(im, (IMAGE_SIZE, IMAGE_SIZE))
#     im = im.astype(np.float32)[:, :, ::-1]
    big_dim = max(im.width, im.height)
    wide = im.width > im.height
    new_w = 224 if not wide else int(im.width * 224 / im.height)
    new_h = 224 if wide else int(im.height * 224 / im.width)
    im = im.resize((new_w, new_h)).crop((0, 0, 224, 224))
    im = (np.asarray(im) / 255.0).astype(np.float32)

    saliency_images = predictor(im)[0]

    abs_saliency = np.abs(saliency_images).max(axis=-1)
    pos_saliency = np.maximum(0, saliency_images)
    neg_saliency = np.maximum(0, -saliency_images)
    
    from skimage.io import imsave

    pos_saliency -= pos_saliency.min()
    pos_saliency /= pos_saliency.max()
    imsave(fname + '_pos.png', pos_saliency, plugin='pil', format_str='png')

    neg_saliency -= neg_saliency.min()
    neg_saliency /= neg_saliency.max()
    imsave(fname + '_neg.png', neg_saliency, plugin='pil', format_str='png')

    abs_saliency = viz.intensity_to_rgb(abs_saliency, normalize=True)[:, :, ::-1]  # bgr
    imsave(fname + '_abs-saliency.png', abs_saliency / 255, plugin='pil', format_str='png')

    rsl = im * 0.2 + abs_saliency * 0.8
    imsave(fname + '_blended.png', rsl / 255, plugin='pil', format_str='png')

INFO:tensorflow:Scale of 0 disables regularizer.
[0324 20:57:50 @collection.py:145] New collections created in tower : tf.GraphKeys.MODEL_VARIABLES, moving_vars
[0324 20:57:50 @sessinit.py:87] WRN The following variables are in the graph, but not found in the checkpoint: inception_v1/mean_rgb
[0324 20:57:50 @sessinit.py:87] WRN The following variables are in the checkpoint, but not found in the graph: global_step
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[0324 20:57:51 @sessinit.py:114] Restoring checkpoint from /home/w266ajh/Documents/inception_v1.ckpt ...
INFO:tensorflow:Restoring parameters from /home/w266ajh/Documents/inception_v1.ckpt


In [6]:
for file in os.listdir("/home/w266ajh/Documents/top5gen/data_imageparts_1"):
    if file == ".directory": continue
    run("", "/home/w266ajh/Documents/top5gen/data_imageparts_1/" + file)

/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/weevil2.jpg_pos.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/weevil2.jpg_neg.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/weevil2.jpg_blended.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/remote2.jpg_pos.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.local/lib/python3.6/site

/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/rule4.jpg_pos.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/rule4.jpg_neg.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/rule4.jpg_abs-saliency.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/rule4.jpg_blended.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/burger1.jpg_pos.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.local/lib/python3.6/

/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/jackfruit3.jpg_pos.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/jackfruit3.jpg_neg.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/schoolbus1.jpg_pos.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/schoolbus1.jpg_neg.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/schoolbus1.jpg_abs-saliency.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.l

/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/schoolbus3.jpg_pos.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/schoolbus3.jpg_neg.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/schoolbus3.jpg_abs-saliency.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/schoolbus3.jpg_blended.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: normal_saliencies/weevil1.jpg_pos.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/w266ajh/.

In [3]:
sess = tf.InteractiveSession()
image = tf.Variable(tf.zeros((224, 224, 3)))

import json
imagenet_json, _ = urlretrieve(
    'http://www.anishathalye.com/media/2017/07/25/imagenet.json')
with open(imagenet_json) as f:
    imagenet_labels = json.load(f)

# Define our custom get_top_five
def get_top_five(path, img):
    # Get predictions
    p = sess.run(probs, feed_dict={image: img})[0]
    topk = list(p.argsort()[-10:][::-1])
    topprobs = p[topk]
    
    # Put results in nice list
    current_results = [path]
    for i in range(5):
        current_results.append('{}: probability: {:.02f}, label: {}'
              .format(i+1, topprobs[i], imagenet_labels[topk[i]]))

    # Legacy requires us to wrap the results in another empty list
    to_return = []
    to_return.append(current_results)
    return(to_return)

def incep(image, reuse):
    preprocessed = tf.multiply(tf.subtract(tf.expand_dims(image, 0), 0.5), 2.0)
    arg_scope = inception.inception_v1_arg_scope(weight_decay=0.0)
    with slim.arg_scope(arg_scope):
        logits, _ = inception.inception_v1(
            preprocessed, 1001, is_training=False, reuse=reuse)
        logits = logits[:,1:] # ignore background class
        probs = tf.nn.softmax(logits) # probabilities
    return logits, probs

logits, probs = incep(image, reuse=False)

restore_vars = [
    var for var in tf.global_variables()
    if var.name.startswith('InceptionV1/')
]
saver = tf.train.Saver(restore_vars)
saver.restore(sess, "/home/w266ajh/Documents/inception_v1.ckpt")

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/w266ajh/Documents/inception_v1.ckpt


In [4]:
# Get image and write out the results
img = PIL.Image.open("/home/w266ajh/Documents/jupyter_notebooks/adv_images/adv_barn1.jpg_5.png")
big_dim = max(img.width, img.height)
wide = img.width > img.height
new_w = 224 if not wide else int(img.width * 224 / img.height)
new_h = 224 if wide else int(img.height * 224 / img.width)
img = img.resize((new_w, new_h)).crop((0, 0, 224, 224))
img = (np.asarray(img) / 255.0).astype(np.float32)

print(get_top_five("img_path", img))

[['img_path', '1: probability: 0.85, label: electric ray, crampfish, numbfish, torpedo', '2: probability: 0.04, label: stingray', '3: probability: 0.01, label: boathouse', '4: probability: 0.01, label: barn', '5: probability: 0.00, label: snail']]
